### Estimating Passenger Outflows Using Gravity Models for Public Transport Stops
This notebook explores the estimation of passenger outflows (alightings) across the Lisbon bus network using gravity-based mobility models. Starting from raw vehicle event and validation data, we reconstruct origin–destination (OD) flows at the stop level and develop a consistent spatial tessellation. Because classical gravity models are sensitive to dimensionality and numerical instability when applied to thousands of micro-stops, we aggregate stops into a reduced set of spatial zones using clustering. This zonal representation enables robust model fitting, avoids zero-distance and collinearity issues, and preserves the underlying mobility structure of the network.

Using these aggregated OD flows, we calibrate a singly constrained gravity model to explain the spatial distribution of passenger movements. If the gravity model cannot converge due to extreme overdispersion or matrix instability, the notebook automatically falls back to the radiation model as a reliable alternative. The resulting predicted OD flows are then disaggregated back to the original bus stops, producing estimated stop-level outflows (exits). This workflow provides a scalable and methodologically sound approach for predicting passenger alighting patterns when direct exit counts are not available.

In [1]:
import skmob
import pandas as pd
import geopandas as gpd
from fiona import path

In [2]:
tessellation = gpd.read_file("data/out/tesselation_clean_clip.geojson").rename(columns={'stop_id': 'tile_ID'})

In [3]:
stops = pd.read_csv("data/stops.csv")

/var/folders/hh/pvwk6s4917qdt0_j791cx6wc0000gn/T/ipykernel_7615/435690302.py:1: DtypeWarning: Columns (50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  stops = pd.read_csv("data/stops.csv")


In [88]:
stops = gpd.GeoDataFrame(stops, geometry=gpd.points_from_xy(stops.stop_lon, stops.stop_lat), crs="EPSG:4326")

# buffer 30m
tessellation = stops.copy()
tessellation["geometry"] = tessellation.to_crs(3857).buffer(30).to_crs(4326)
tessellation = tessellation.drop_duplicates(subset=["stop_id"])
tessellation = tessellation.rename(columns={"stop_id": "tile_ID"})
tessellation = tessellation[["tile_ID", "geometry"]]


In [4]:
tessellation = tessellation.drop_duplicates(subset=["tile_ID"])

In [11]:
routes = pd.read_csv("data/out/routes.csv")

/var/folders/hh/pvwk6s4917qdt0_j791cx6wc0000gn/T/ipykernel_7615/3994241183.py:1: DtypeWarning: Columns (23,40,43,44,66,67) have mixed types. Specify dtype option on import or set low_memory=False.
  routes = pd.read_csv("data/out/routes.csv")


In [5]:
flows = pd.read_csv("data/out/flows_first3_pairs_line1_nodes_041125.csv")

In [6]:
flows = flows.rename(columns={"from_stop_id": "origin", "to_stop_id": "destination", "num_pairs": "flow"})

In [7]:
flows

,from_line_id,origin,destination,from_node,to_node,num_trips,flow
0,3013,29589,20682,29589__3013,20682__3013,181,192
1,3021,20563,21017,20563__3021,21017__3021,148,186
2,1710,30870,60071,30870__1710,60071__1710,145,162
3,2224,110069,110097,110069__2224,110097__2224,144,160
4,2126,80107,80007,80107__2126,80007__2126,144,194
...,...,...,...,...,...,...,...
78283,2750,70555,110623,70555__2750,110623__2750,1,1
78284,2750,70558,71133,70558__2750,71133__2750,1,1
78285,1625,172341,170137,172341__1625,170137__1625,1,1
78286,2750,70558,110801,70558__2750,110801__2750,1,1


In [12]:
routes = gpd.GeoDataFrame(routes, geometry=gpd.points_from_xy(routes.stop_lon, routes.stop_lat), crs="EPSG:4326")

In [13]:
routes["stop_id"] = routes["stop_id"].astype(str)
routes["vehicle_id"] = routes["vehicle_id"].astype(str)
routes["trip_id"] = routes["trip_id"].astype(str)

# --- 1) Sort & build next_stop transitions -----------------------------------
routes = routes.sort_values(["vehicle_id", "trip_id", "created_at"])
routes["next_stop"] = routes.groupby(
    ["vehicle_id", "trip_id"], group_keys=False
)["stop_id"].shift(-1)


In [20]:
transitions = routes.dropna(subset=["next_stop"]).copy()

# Ensure IDs are strings
transitions["stop_id"] = transitions["stop_id"].astype(str)
transitions["next_stop"] = transitions["next_stop"].astype(str)

# Compute OD flows by **summing num_validations**
flow_df = (
    transitions.groupby(["stop_id", "next_stop"])["num_validations"]
    .sum()
    .reset_index()
    .rename(columns={
        "stop_id": "origin",
        "next_stop": "destination",
        "num_validations": "flow"
    })
)

In [22]:
flow_df_original = flow_df

In [8]:
flow_df = flows[["origin", "destination", "flow"]]

In [9]:
flow_df["origin"] = flow_df["origin"].astype(str)
flow_df["destination"] = flow_df["destination"].astype(str)

/var/folders/hh/pvwk6s4917qdt0_j791cx6wc0000gn/T/ipykernel_7615/1073262273.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flow_df["origin"] = flow_df["origin"].astype(str)
/var/folders/hh/pvwk6s4917qdt0_j791cx6wc0000gn/T/ipykernel_7615/1073262273.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flow_df["destination"] = flow_df["destination"].astype(str)


In [14]:
# --- 2) Time-of-day weighted RELEVANCE per stop ------------------------------
routes["created_at"] = pd.to_datetime(routes["created_at"], errors="coerce")
# Your binary/indicator feature columns
cols_features = [
    'near_health_clinic', 'near_hospital', 'near_university', 'near_school',
    'near_police_station', 'near_fire_station', 'near_shopping',
    'near_historic_building', 'near_transit_office', 'near_beach', 'subway',
    'light_rail', 'train', 'boat', 'airport', 'bike_sharing',
    'bike_parking', 'car_parking'
]

# Make sure they exist; if not, create zeros
for c in cols_features:
    if c not in routes.columns:
        routes[c] = 0

# Define 4 time bands (tweak as you like)
def time_band(h):
    if 6 <= h < 10:   # Morning peak
        return "morning"
    if 10 <= h < 16:  # Midday
        return "midday"
    if 16 <= h < 20:  # Evening peak
        return "evening"
    return "night"    # Night

routes["hour"] = routes["created_at"].dt.hour.fillna(0).astype(int)
routes["band"] = routes["hour"].apply(time_band)

# Base weight (importance) for each feature (1.0 = neutral)
base_w = {
    'near_health_clinic': 1.1, 'near_hospital': 1.3, 'near_university': 1.2,
    'near_school': 1.2, 'near_police_station': 1.0, 'near_fire_station': 1.0,
    'near_shopping': 1.2, 'near_historic_building': 1.0,
    'near_transit_office': 1.0, 'near_beach': 1.1, 'subway': 1.2,
    'light_rail': 1.2, 'train': 1.2, 'boat': 1.0, 'airport': 1.2,
    'bike_sharing': 1.0, 'bike_parking': 1.0, 'car_parking': 1.0,
}

# Time-of-day multipliers per feature (domain heuristics; adjust for Lisbon)
time_w = {
    "morning": {  # commute + schools
        'near_school': 1.5, 'near_university': 1.3,
        'subway': 1.3, 'light_rail': 1.3, 'train': 1.3,
        'airport': 1.1, 'near_hospital': 1.1
    },
    "midday": {  # errands, shopping, services
        'near_shopping': 1.4, 'near_transit_office': 1.2,
        'near_university': 1.2, 'near_health_clinic': 1.2,
        'near_hospital': 1.2, 'near_historic_building': 1.2, 'beach': 1.0
    },
    "evening": {  # return trips, leisure
        'near_shopping': 1.4, 'near_beach': 1.4,
        'subway': 1.2, 'light_rail': 1.2, 'train': 1.2,
        'bike_sharing': 1.1, 'car_parking': 1.1
    },
    "night": {   # emergencies, airport, parking
        'near_hospital': 1.5, 'near_police_station': 1.3,
        'airport': 1.3, 'car_parking': 1.2
    }
}

# Build a per-row multiplier vectorized:
# Start from base weights (same for all rows)
base_arr = routes[cols_features].mul(
    pd.Series(base_w).reindex(cols_features).fillna(1.0), axis=1
)

# Build time-of-day multiplier per row per feature
def row_time_mult(band):
    # default 1.0 for all features, then override with time_w[band]
    d = {c: 1.0 for c in cols_features}
    for k, v in time_w.get(band, {}).items():
        if k in d: d[k] = v
    return pd.Series(d)

time_mult_df = routes["band"].map(row_time_mult)
# map() over Series returns Series of dicts; convert properly:
time_mult_df = pd.DataFrame(list(time_mult_df)).reindex(columns=cols_features).fillna(1.0)

# Final per-row relevance = sum(features * base * time_multiplier)
routes["relevance"] = (routes[cols_features] * base_arr * time_mult_df).sum(axis=1)

# Aggregate to stop-level "attraction" (destination pull)
stop_relevance = (
    routes.groupby("stop_id", as_index=False)["relevance"]
    .sum()
    .rename(columns={"stop_id": "tile_id", "relevance": "attraction_relevance"})
)

# --- 3) What you have now -----------------------------------------------------
#  - flow_df: OD transitions with counts (origin, destination, flow)
#  - stop_relevance: per-stop attraction score, already time-weighted and summed

# Example: join relevance onto destinations if you want a decorated OD table
flow_with_dest_attr = flow_df.merge(
    stop_relevance.rename(columns={"tile_id": "destination"}),
    on="destination",
    how="left"
)

# If you also want origin-side "production" relevance:
origin_relevance = stop_relevance.rename(columns={
    "tile_id": "origin", "attraction_relevance": "origin_relevance"
})
flow_with_attrs = flow_with_dest_attr.merge(origin_relevance, on="origin", how="left")

In [15]:
stop_relevance = stop_relevance.rename(columns={"tile_id": "tile_ID"})

In [16]:
tessellation["tile_ID"] = tessellation["tile_ID"].astype(str)
flow_df["origin"] = flow_df["origin"].astype(str)
flow_df["destination"] = flow_df["destination"].astype(str)

# (optional) attach your destination “attraction” to the tessellation
#    rename your relevance column to something explicit, e.g. 'attraction_relevance'
#    stop_relevance: columns ['tile_id','attraction_relevance']
tessellation = tessellation.merge(
    stop_relevance, on="tile_ID", how="left"
).fillna({"attraction_relevance": 0.0})

/var/folders/hh/pvwk6s4917qdt0_j791cx6wc0000gn/T/ipykernel_7615/1973544299.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flow_df["origin"] = flow_df["origin"].astype(str)
/var/folders/hh/pvwk6s4917qdt0_j791cx6wc0000gn/T/ipykernel_7615/1973544299.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flow_df["destination"] = flow_df["destination"].astype(str)


In [17]:
tot_outflows = flow_df[flow_df['origin'] != flow_df['destination']]

In [18]:
tot_outflows = tot_outflows.nlargest(1000, "flow")

In [19]:
tot_outflows

,origin,destination,flow
4,80107,80007,194
0,29589,20682,192
1,20563,21017,186
2,30870,60071,162
5,71364,70514,161
...,...,...,...
1600,120997,120945,19
1639,70096,72692,19
1642,60226,70835,19
1682,100287,100013,19


In [20]:
tessellation = tessellation.rename(columns={"attraction_relevance": "relevance"})

In [41]:
from skmob.utils import utils, constants
tessellation_2 = tessellation.merge(flow_df, left_on='tile_ID', right_on='origin').rename(columns={'flow': constants.TOT_OUTFLOW})

In [115]:
from skmob.core.flowdataframe import FlowDataFrame as FDF
from skmob.models.gravity import Gravity

# Sanity print before FDF (you should see GeoDataFrame + Polygon/MultiPolygon)
# diagnose_tessellation(tessellation_gdf, id_col="tile_id")

fdf = FDF(
    tot_outflows,
    origin="origin",
    destination="destination",
    flow="flow",
    tessellation=tessellation,   # must be GeoDataFrame with tile_id + geometry
    tile_id="tile_ID",
)

In [116]:
fdf

,origin,destination,flow
4,80107,80007,194
0,29589,20682,192
1,20563,21017,186
2,30870,60071,162
5,71364,70514,161
...,...,...,...
1600,120997,120945,19
1639,70096,72692,19
1642,60226,70835,19
1682,100287,100013,19


In [ ]:
gravity = Gravity(gravity_type="singly constrained")
gravity.fit(fdf, relevance_column="relevance")   # or relevance_column="attraction_relevance" if you merged it

In [ ]:
pred = gravity.generate(tessellation_2,tile_id_column='tile_ID',tot_outflows_column='tot_outflow',
                        relevance_column= 'relevance',
                        out_format='flows')

In [30]:
from skmob.models.gravity import Gravity
gravity_singly = Gravity(gravity_type='singly constrained')
synth_fdf = gravity_singly.generate(tessellation_2,
                                   tile_id_column='tile_ID',
                                   tot_outflows_column='tot_outflow',
                                   relevance_column= 'relevance',
                                   out_format='flows')

100%|██████████| 1000/1000 [00:00<00:00, 2193.44it/s]
/Users/kfilonchik/anaconda3/envs/skmob_2/lib/python3.8/site-packages/skmob/models/gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)
/Users/kfilonchik/anaconda3/envs/skmob_2/lib/python3.8/site-packages/skmob/models/gravity.py:298: RuntimeWarning: invalid value encountered in multiply
  trip_probs_matrix = trip_probs_matrix * relevances_dest ** self.destination_exp * \
/Users/kfilonchik/anaconda3/envs/skmob_2/lib/python3.8/site-packages/skmob/models/gravity.py:378: RuntimeWarning: invalid value encountered in divide
  trip_probs_matrix = (trip_probs_matrix.T / np.sum(trip_probs_matrix, axis=1)).T


In [32]:
print("Fitted parameters:")
print(f"  Origin exponent:      {gravity_singly.origin_exp:.4f}")
print(f"  Destination exponent: {gravity_singly.destination_exp:.4f}")
print(f"  Deterrence type:      {gravity_singly.deterrence_func_type}")
print(f"  Deterrence args:      {gravity_singly.deterrence_func_args}")  # e.g. [lambda] or [alpha]


Fitted parameters:
  Origin exponent:      1.0000
  Destination exponent: 1.0000
  Deterrence type:      power_law
  Deterrence args:      [-2.0]


In [31]:
synth_fdf

,origin,destination,flow
0,20835,20837,2.975331
1,20835,20661,0.071486
2,20835,20458,0.063748
3,20835,20458,0.063748
4,20835,20471,0.017705
...,...,...,...
561889,100061,180787,0.005632
561890,100061,180787,0.005632
561891,100061,100006,1.913968
561892,100061,100006,1.913968


In [43]:
tessellation_2

,OBJECTID,Input_FID,tile_ID,stop_name,stop_name_new,stop_short_name,stop_lat,stop_lon,operational_status,areas,...,bike_sharing,bike_parking,car_parking,Shape_Length,Shape_Area,geometry,relevance,origin,destination,tot_outflow
0,3,948,29802,FONTE TELHA (ESTR F TELHA 7),a definir,a definir,38.636076,-9.228236,ACTIVE,43,...,0,0,0,1851.396624,1.322792e+05,"POLYGON ((-9.22199 38.64027, -9.22323 38.63992...",0.0,29802,20002,3
1,3,948,29802,FONTE TELHA (ESTR F TELHA 7),a definir,a definir,38.636076,-9.228236,ACTIVE,43,...,0,0,0,1851.396624,1.322792e+05,"POLYGON ((-9.22199 38.64027, -9.22323 38.63992...",0.0,29802,20460,2
2,3,948,29802,FONTE TELHA (ESTR F TELHA 7),a definir,a definir,38.636076,-9.228236,ACTIVE,43,...,0,0,0,1851.396624,1.322792e+05,"POLYGON ((-9.22199 38.64027, -9.22323 38.63992...",0.0,29802,20661,1
3,4,472,20441,AV GENERAL HUMBERTO DELGADO (LOTA COSTA),a definir,a definir,38.638494,-9.233924,ACTIVE,43,...,0,0,0,1146.324190,5.213228e+04,"POLYGON ((-9.23462 38.63960, -9.23243 38.63547...",0.0,20441,20456,2
4,4,472,20441,AV GENERAL HUMBERTO DELGADO (LOTA COSTA),a definir,a definir,38.638494,-9.233924,ACTIVE,43,...,0,0,0,1146.324190,5.213228e+04,"POLYGON ((-9.23462 38.63960, -9.23243 38.63547...",0.0,20441,20453,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78282,12644,5243,100261,TAIPADAS (EN 10) CASA CANTONEIROS,a definir,a definir,38.746828,-8.664236,ACTIVE,44,...,0,0,0,2170.810175,2.516824e+05,"POLYGON ((-8.66648 38.74768, -8.66893 38.74725...",0.0,100261,100281,1
78283,12645,5350,100383,FIGUEIRAS (EN 4) MARCO 37,a definir,a definir,38.683480,-8.692437,ACTIVE,44,...,0,0,0,9897.443777,3.884463e+06,"POLYGON ((-8.69044 38.70445, -8.69393 38.70289...",0.0,100383,100041,3
78284,12645,5350,100383,FIGUEIRAS (EN 4) MARCO 37,a definir,a definir,38.683480,-8.692437,ACTIVE,44,...,0,0,0,9897.443777,3.884463e+06,"POLYGON ((-8.69044 38.70445, -8.69393 38.70289...",0.0,100383,130139,2
78285,12676,5191,100201,CANHA (R CASTELO) TERMINAL,a definir,a definir,38.768191,-8.626242,ACTIVE,44,...,0,0,0,9490.965285,4.416472e+06,"POLYGON ((-8.63116 38.76783, -8.61605 38.76863...",18.0,100201,100238,3


In [48]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from shapely.geometry import Point, LineString
from branca.colormap import LinearColormap

# ----------------------------
# 1) Compute exits per tile
# ----------------------------

# Merge onto tessellation
tess = tessellation_2[["tile_ID","geometry", "tot_outflow"]].copy()
tess["tile_id"] = tess["tile_ID"].astype(str)
tess["predicted_exits"] = tess["tot_outflow"].fillna(0.0)

# ----------------------------
# 2) Prepare centroids (for lines)
# ----------------------------
cent = tess.copy()
cent["centroid"] = cent.geometry.centroid
cent_coords = pd.DataFrame({
    "tile_id": cent["tile_id"],
    "lat": cent["centroid"].y,
    "lon": cent["centroid"].x
})

# ----------------------------
# 3) Build a color scale
# ----------------------------
vmin = float(tess["predicted_exits"].min())
vmax = float(tess["predicted_exits"].max())
# A readable blue scale: light (low) -> dark (high)
cmap = LinearColormap(
    colors=["#eff3ff","#bdd7e7","#6baed6","#3182bd","#08519c"],
    vmin=vmin, vmax=vmax
)
cmap.caption = "Predicted exits (arrivals)"

# ----------------------------
# 4) Folium map centered on Lisbon tessellation
# ----------------------------
# center by polygon bounds
cx = tess.geometry.centroid.x.mean()
cy = tess.geometry.centroid.y.mean()
m = folium.Map(location=[cy, cx], zoom_start=12, tiles="cartodbpositron")

# Layer: Exits choropleth
def style_fn(feature):
    val = feature["properties"].get("predicted_exits", 0.0)
    color = "#4d8302" if val is None else cmap(val)
    return {
        "fillColor": color,
        "color": "#0E3677",
        "weight": 0.5,
        "fillOpacity": 0.75,
    }

gj = folium.GeoJson(
    tess.to_json(),
    name="Exits by tessellation",
    style_function=style_fn,
    highlight_function=lambda f: {"weight": 1.5, "color": "#000000"},
    tooltip=folium.GeoJsonTooltip(
        fields=["tile_id","predicted_exits"],
        aliases=["Tile ID","Predicted exits"],
        localize=True
    )
)
gj.add_to(m)
cmap.add_to(m)

# ----------------------------
# 5) Add top OD flow lines
# ----------------------------
# Keep only positive flows and join origin/destination centroids
flows = flow_df[flow_df["flow"] > 0].copy()
flows["origin"] = flows["origin"].astype(str)
flows["destination"] = flows["destination"].astype(str)

flows = (flows
         .merge(cent_coords.rename(columns={"tile_id":"origin",
                                            "lat":"olat","lon":"olon"}), on="origin", how="inner")
         .merge(cent_coords.rename(columns={"tile_id":"destination",
                                            "lat":"dlat","lon":"dlon"}), on="destination", how="inner"))

# Focus on strongest flows so the map is readable
TOP_K = 300  # adjust
flows_top = flows.sort_values("flow", ascending=False).head(TOP_K).copy()

# Scale line width between min_w and max_w
min_w, max_w = 1.0, 8.0
flow_min, flow_max = float(flows_top["flow"].min()), float(flows_top["flow"].max())
def width_scale(v):
    if flow_max == flow_min:
        return (min_w + max_w) / 2
    return min_w + (max_w - min_w) * ((v - flow_min) / (flow_max - flow_min))

# Optional: reuse same colormap to color lines by magnitude
flows_fg = folium.FeatureGroup(name=f"Top {TOP_K} OD flows", show=False)
for _, r in flows_top.iterrows():
    line = LineString([(r["olon"], r["olat"]), (r["dlon"], r["dlat"])])
    val = float(r["flow"])
    folium.PolyLine(
        locations=[(r["olat"], r["olon"]), (r["dlat"], r["dlon"])],
        color=cmap(val),
        weight=width_scale(val),
        opacity=0.7,
        tooltip=f"OD: {r['origin']} → {r['destination']} | flow={int(val)}"
    ).add_to(flows_fg)
flows_fg.add_to(m)

# ----------------------------
# 6) Layer control & save
# ----------------------------
folium.LayerControl(collapsed=False).add_to(m)
m.save("lisbon_flows_exits.html")
print("Map saved to lisbon_flows_exits.html")


/var/folders/hh/pvwk6s4917qdt0_j791cx6wc0000gn/T/ipykernel_7615/97466889.py:21: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cent["centroid"] = cent.geometry.centroid
/var/folders/hh/pvwk6s4917qdt0_j791cx6wc0000gn/T/ipykernel_7615/97466889.py:44: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cx = tess.geometry.centroid.x.mean()
/var/folders/hh/pvwk6s4917qdt0_j791cx6wc0000gn/T/ipykernel_7615/97466889.py:45: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cy = tess.geometry.centroid.y.mean()


Map saved to lisbon_flows_exits.html
